# Import Libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# importing my df_products_merged_with_orders2 as df_new_products
df_new_products = pd.read_pickle(r'C:\Users\Babafemi Olayinka\Jupyter Notebook\FEB-2023 INSTACART BASKET ANALYSIS\Data\Prepared Data\df_products_merged_with_orders2.pkl')

In [3]:
df_new_products.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,eval_set,Busiest day,Busiest days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5.0,0.0,both,NaN,Regularly busy,Regular days,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1.0,1.0,both,NaN,Regularly busy,Regular days,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20.0,0.0,both,NaN,Busiest day,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,7.0,10.0,0.0,both,NaN,Regularly busy,Slowest days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11.0,1.0,both,NaN,Least busy,Slowest days,Average orders


In [5]:
df_new_products.shape

(32434212, 18)

In [7]:
# creating a subset of df_new_products where it will only contain the first 1 Million rows [:1000000] named df_new_prooducts_merged
df_new_products_merged_orders = df_new_products[:1000000]

In [8]:
# Dimension
df_new_products_merged_orders.shape

(1000000, 18)

In [9]:
df_new_products_merged_orders.head(2)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,eval_set,Busiest day,Busiest days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5.0,0.0,both,NaN,Regularly busy,Regular days,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1.0,1.0,both,NaN,Regularly busy,Regular days,Average orders


In [10]:
# using the groupby fx to group my df by it product name column
df_new_products_merged_orders.groupby('product_name')

# What has pandas done here? Well, the groupby() function has apparently created a pandas object (indicated by the term “object” in the output message above). But now what? Nothing seems to have actually happened

# Aggregating Data with agg(), while using a single aggregation

## the average number of orders per department ID below . You can easily see, for instance, that the department with an ID of 4 (produce) has a mean of around 19. Conversely, the department with an ID of 17 (household), only has a mean of around 11. The insight you can glean from this, then, is that produce sells considerably more on average than household goods

In [13]:
# determining which dept is the best selling dept by using the aggregate mean of the order_number in each dept_id
df_new_products_merged_orders.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.562844
2,17.518895
3,16.907152
4,17.537934
5,14.894521
6,16.658449
7,17.072954
8,14.942021


In [14]:
# performing aggregation, without the use of agg() fx. such as calc the mean.
df_new_products_merged_orders.groupby('department_id')['order_number'].mean()

department_id
1     15.562844
2     17.518895
3     16.907152
4     17.537934
5     14.894521
6     16.658449
7     17.072954
8     14.942021
9     15.447580
10    18.681852
11    15.352278
12    14.327957
13    16.578690
14    16.666369
15    16.153766
16    17.768070
17    15.602964
18    19.674252
19    16.924344
20    16.292631
21    25.535596
Name: order_number, dtype: float64

## Performing multiple aggregation

In [15]:
# performing multiple descriptive statistics at once using the agg() and grouping by a column.
df_new_products_merged_orders.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.562844   1  99
2                17.518895   1  96
3                16.907152   1  99
4                17.537934   1  99
5                14.894521   1  99
6                16.658449   1  99
7                17.072954   1  99
8                14.942021   1  98
9                15.447580   1  99
10               18.681852   1  99
11               15.352278   1  99
12               14.327957   1  99
13               16.578690   1  99
14               16.666369   1  99
15               16.153766   1  99
16               17.768070   1  99
17               15.602964   1  99
18               19.674252   1  99
19               16.924344   1  99
20               16.292631   1  99
21               25.535596   1  98

## Aggregating data with transform() to create a flag column. 
Flags help you target specific groups within your data according to a set of conditions, allowing for segmented analysis

## creating a flag for customers with many orders, after which you must have set your criteria. To achieve this; 3 steps to get the result are 
1- Split the data into groups based on the “user_id” column
2- Apply the transform() function on the “order_number” column to generate the maximum orders for each user
3- Create a new column, “max_order,” into which you’ll place the results of your aggregation

In [16]:
# the 3 steps are in the code;new column max_order, groupby the user_id and transform order_no with the max() in Np Library 
df_new_products_merged_orders['max_order'] = df_new_products_merged_orders.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Babafemi Olayinka\AppData\Local\Temp\ipykernel_18792\1376305920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_products_merged_orders['max_order'] = df_new_products_merged_orders.groupby(['user_id'])['order_number'].transform(np.max)


In [19]:
# to view the newly created column max_order
df_new_products_merged_orders.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,eval_set,Busiest day,Busiest days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5.0,0.0,both,NaN,Regularly busy,Regular days,Most orders,30
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1.0,1.0,both,NaN,Regularly busy,Regular days,Average orders,30
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20.0,0.0,both,NaN,Busiest day,Busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,7.0,10.0,0.0,both,NaN,Regularly busy,Slowest days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11.0,1.0,both,NaN,Least busy,Slowest days,Average orders,3


## Deriving columns with loc(). Now you have created the new column you wanted.
Next is to set that criteria to perform your segmented analysis ( which is to see which customer using the max_order new column
has a regular customer status, loyal customer status and new customer status0.

In [20]:
df_new_products_merged_orders.loc[df_new_products_merged_orders['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_new_products_merged_orders.loc[(df_new_products_merged_orders['max_order'] <= 40) & (df_new_products_merged_orders['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_new_products_merged_orders.loc[df_new_products_merged_orders['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

C:\Users\Babafemi Olayinka\AppData\Local\Temp\ipykernel_18792\2870952121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_products_merged_orders.loc[df_new_products_merged_orders['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [22]:
# setting the criteria that a new column loyalty_flag created should show a string value of 'loyal customer' if the 
max_order for that row is > 40 within my df
df_new_products_merged_orders.loc[df_new_products_merged_orders['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [23]:
# setting the criteria that a new column loyalty_flag created should show a string value of 'Regular customer' if the 
max_order for that row is <= 40 and > 10 within my df
df_new_products_merged_orders.loc[(df_new_products_merged_orders['max_order'] <= 40) & (df_new_products_merged_orders['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [24]:
# setting the criteria that a new column loyalty_flag created should show a string value of 'New customer' if the 
max_order for that row is < 10 within my df
df_new_products_merged_orders.loc[df_new_products_merged_orders['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [1]:
df_new_products_merged_orders['loyalty_flag'].value_counts(dropna = False)

NameError: name 'df_new_products_merged_orders' is not defined

# Taking care of outliers.
This was achieved in 4.9 Exercise. We noticed some price in the price column are outliers after using scatterplot to view the
distribution. FFG is the code to use.

In [3]:
# marking any value above 100 dollar within the price column missing values Nan
df_new_products_merged_orders[df_new_products_merged_orders['prices'] > 100]

NameError: name 'df_new_products_merged_orders' is not defined

In [ ]:
df_new_products_merged_orders[df_new_products_merged_orders['prices'] >100, 'prices'] = np.nan